In [1]:
import pandas as pd
import plotly.graph_objects as go

# Load your Excel data
file_path = 'Todays Python Workflow.xlsx'  # Replace with your path
df = pd.read_excel(file_path)

# Filter out rows where Qty equals 0
df = df[df['Qty'] != 0]

# Extract expiration dates from the 'Option' column and convert them to datetime format
df['Expiration'] = pd.to_datetime(df['Option'].str.extract(r'(\d{1,2} \w{3} \d{2})')[0], format='%d %b %y', errors='coerce')

# Ensure the Premium is calculated correctly (Qty * Price * 100)
df['Premium'] = df['Qty'] * df['Price'] * 100

# Sort the data by Expiration to determine "today" and "tomorrow"
df_sorted = df.sort_values(by='Expiration').reset_index(drop=True)

# Identify the first two unique expiration dates (today and tomorrow)
unique_expirations = df_sorted['Expiration'].dropna().unique()

if len(unique_expirations) >= 2:
    today_expiration = unique_expirations[0]  # "Today" is the first date
    tomorrow_expiration = unique_expirations[1]  # "Tomorrow" is the second date
else:
    today_expiration = unique_expirations[0] if len(unique_expirations) == 1 else None
    tomorrow_expiration = None

# Filter the data for 0DTE (today) and 1DTE (tomorrow)
df_0dte_1dte = df_sorted[(df_sorted['Expiration'] == today_expiration) | (df_sorted['Expiration'] == tomorrow_expiration)]

# Clean up Bot/Sold column by stripping extra spaces and converting to uppercase using .loc[] to avoid warning
df_0dte_1dte.loc[:, 'Bot/Sold'] = df_0dte_1dte['Bot/Sold'].str.strip().str.upper()

# Separate the data for graph (excluding 'MID') and for total premium calculations (including 'MID')
df_graph = df_0dte_1dte[df_0dte_1dte['Bot/Sold'].isin(['BOT', 'SOLD'])].copy()  # Using .copy() to avoid warnings
df_total = df_0dte_1dte.copy()  # Includes 'MID' for total premium calculations

# Classify trades into Bot/Sold Calls and Puts using .loc[] to avoid SettingWithCopyWarning
def classify_trades(dataframe):
    dataframe.loc[:, 'Bot Call'] = dataframe.apply(lambda row: row['Premium'] if row['Type'] == 'Call' and row['Bot/Sold'] == 'BOT' else 0, axis=1)
    dataframe.loc[:, 'Sold Call'] = dataframe.apply(lambda row: row['Premium'] if row['Type'] == 'Call' and row['Bot/Sold'] == 'SOLD' else 0, axis=1)
    dataframe.loc[:, 'Bot Put'] = dataframe.apply(lambda row: row['Premium'] if row['Type'] == 'Put' and row['Bot/Sold'] == 'BOT' else 0, axis=1)
    dataframe.loc[:, 'Sold Put'] = dataframe.apply(lambda row: row['Premium'] if row['Type'] == 'Put' and row['Bot/Sold'] == 'SOLD' else 0, axis=1)
    return dataframe

# Classify trades for graph (excluding 'MID')
df_graph = classify_trades(df_graph)

# Classify trades for total calculation (including 'MID')
df_total = classify_trades(df_total)

# Check if 'Mid Calls Premium' and 'Mid Puts Premium' columns exist and contain data
if 'Mid Call Premium' in df_total.columns:
    total_mid_calls_premium = df_total['Mid Call Premium'].sum()
else:
    total_mid_calls_premium = 0

if 'Mid Put Premium' in df_total.columns:
    total_mid_puts_premium = df_total['Mid Put Premium'].sum()
else:
    total_mid_puts_premium = 0

# Calculate the sum of all premiums (Bot, Sold, and Mid)
total_premium = (
    df_total['Premium'].sum() + total_mid_calls_premium + total_mid_puts_premium
)

# Calculate the total Call and Put premiums (including Bot, Sold, and Mid)
total_call_premium = df_total['Bot Call'].sum() + df_total['Sold Call'].sum() + total_mid_calls_premium
total_put_premium = df_total['Bot Put'].sum() + df_total['Sold Put'].sum() + total_mid_puts_premium

# Calculate the percentage of Mid Calls and Mid Puts Premium of the total premium
mid_calls_percentage_total = (total_mid_calls_premium / total_premium) * 100 if total_premium != 0 else 0
mid_puts_percentage_total = (total_mid_puts_premium / total_premium) * 100 if total_premium != 0 else 0

# Calculate the percentage of Mid Calls and Mid Puts Premium of the total Call/Put premium
mid_calls_percentage_call = (total_mid_calls_premium / total_call_premium) * 100 if total_call_premium != 0 else 0
mid_puts_percentage_put = (total_mid_puts_premium / total_put_premium) * 100 if total_put_premium != 0 else 0

# Group the data by expiration date and sum the values for each category for graph
grouped_data = df_graph.groupby('Expiration').agg({
    'Bot Call': 'sum',
    'Sold Call': 'sum',
    'Bot Put': 'sum',
    'Sold Put': 'sum'
}).reset_index()

# Create a bar chart with Bot Call (light blue), Sold Call (light red), Bot Put (light red), and Sold Put (light blue)
fig = go.Figure()

# Add bars for Bot Calls (light blue)
fig.add_trace(go.Bar(
    x=grouped_data['Expiration'],
    y=grouped_data['Bot Call'],
    name='Bot Call',
    marker_color='blue',
    offsetgroup=0
))

# Add bars for Sold Calls (light red)
fig.add_trace(go.Bar(
    x=grouped_data['Expiration'],
    y=grouped_data['Sold Call'],
    name='Sold Call',
    marker_color='lightcoral',
    offsetgroup=1
))

# Add bars for Bot Puts (light red)
fig.add_trace(go.Bar(
    x=grouped_data['Expiration'],
    y=grouped_data['Bot Put'],
    name='Bot Put',
    marker_color='red',
    offsetgroup=2
))

# Add bars for Sold Puts (light blue)
fig.add_trace(go.Bar(
    x=grouped_data['Expiration'],
    y=grouped_data['Sold Put'],
    name='Sold Put',
    marker_color='lightblue',
    offsetgroup=3
))

# Add the percentage annotations as a second "legend-like" box
annotations_text = (
    f"Mid Calls (of total): {mid_calls_percentage_total:.2f}%<br>"
    f"Mid Puts (of total): {mid_puts_percentage_total:.2f}%<br>"
    f"Mid Calls (of Call): {mid_calls_percentage_call:.2f}%<br>"
    f"Mid Puts (of Put): {mid_puts_percentage_put:.2f}%"
)

fig.add_annotation(
    text=annotations_text,
    align='left',
    showarrow=False,
    xref="paper",
    yref="paper",
    x=0.50,  # Adjust to position to the right of the graph
    y=1.25,
    bordercolor="white",
    borderwidth=1,
    bgcolor="black",
    opacity=0.8
)

# Update the layout
fig.update_layout(
    title='Premium by Expiring Contracts (0DTE and 1DTE)',
    xaxis_title='Expiration Date',
    yaxis_title='Premium Amount',
    barmode='group',  # Group the columns together
    template='plotly_dark'
)

# Show the chart
fig.show()
